In [ ]:
import os, sys
import numpy as np
import pandas as pd

import keras
from keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D,Dense,Dropout,Bidirectional,TimeDistributed,LeakyReLU
from keras.models import Model,Sequential
from keras import optimizers
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
PROJECT_FOLDER = "/content/drive/My Drive/Datasets/KEmoCon/e4_data/"
COMBINED_RAW_DATA_FOLDER = "/content/drive/My Drive/Rare Emotion Recognition/"
AFFECTIVE_DATA_FOLDER = "/content/drive/My Drive/Affective Datasets/"
AFFECTIVE_EMO_DATA_FOLDER = AFFECTIVE_DATA_FOLDER + 'Emotion/'
os.chdir(AFFECTIVE_EMO_DATA_FOLDER)

freq = {'EDA': 4, 'BVP': 64, 'TEMP': 4, 'LABEL': 700}
params = ['EDA','BVP','TEMP']

data_cols = ['id', 'file_location', 'startRowIndex', 'endRowIndex', 'label', 'pid']


In [ ]:
 info_df = pd.read_csv(AFFECTIVE_DATA_FOLDER + 'Affective_datasets_info.csv')

In [ ]:
for param in params:
  param_df = pd.read_csv(AFFECTIVE_EMO_DATA_FOLDER + param + "_raw.csv") #took EDA_raw
  param_df['label'] = 0
  for index, row in param_df.iterrows():
    if (row['angry'] > 1 or row['sad'] > 1 or row['nervous'] > 1):
      param_df.at[index , 'label'] = 1
 
  i,j = 0,0
  idx = info_df.shape[0]
  max_len = param_df.shape[0]
  while i < max_len and j < max_len:
    lbl_i, pid_i = param_df['label'][i],param_df['pid'][i]
    while j < max_len and param_df['label'][j] == lbl_i and param_df['pid'][j] == pid_i:
      j = j + 1
    
    new_row = {'id': idx, 'file_location': 'Emotion/' + param + "_raw.csv", 'startRowIndex':i, 'endRowIndex':(j-1), 'label': int(lbl_i), 'pid': 'Emotion_'+ str(int(pid_i))}
    info_df = info_df.append(new_row, ignore_index=True)
    idx = idx + 1
    i = j

In [ ]:
info_df

,id,file_location,startRowIndex,endRowIndex,label,pid
0,0,Stress1/EDA_2.csv,0,4575,1,Stress1_2
1,1,Stress1/BVP_2.csv,0,73215,1,Stress1_2
2,2,Stress1/TEMP_2.csv,0,4575,1,Stress1_2
3,3,Stress1/EDA_3.csv,0,4559,1,Stress1_3
4,4,Stress1/BVP_3.csv,0,72959,1,Stress1_3
...,...,...,...,...,...,...
640,640,Emotion/TEMP_raw.csv,75132,75211,1,Emotion_31
641,641,Emotion/TEMP_raw.csv,75212,75251,0,Emotion_31
642,642,Emotion/TEMP_raw.csv,75252,75291,1,Emotion_31
643,643,Emotion/TEMP_raw.csv,75292,75373,0,Emotion_31


In [ ]:
info_df.to_csv(AFFECTIVE_DATA_FOLDER + 'Affective_datasets_info.csv', index = False) 

In [ ]:
####END

In [ ]:
param

'EDA'

In [ ]:
  param_df.head

<bound method NDFrame.head of           timestamp  pid     value device_serial  ...  eureka  pride  sorrow  none_2
0      1.548121e+12    1  0.720992        A01A3A  ...       0      0       0     1.0
1      1.548121e+12    1  0.720713        A01A3A  ...       0      0       0     1.0
2      1.548121e+12    1  0.720341        A01A3A  ...       0      0       0     1.0
3      1.548121e+12    1  0.719225        A01A3A  ...       0      0       0     1.0
4      1.548121e+12    1  0.718853        A01A3A  ...       0      0       0     1.0
...             ...  ...       ...           ...  ...     ...    ...     ...     ...
80632  1.552896e+12   28  0.016917        A01525  ...       0      0       0     1.0
80633  1.552896e+12   28  0.016638        A01525  ...       0      0       0     1.0
80634  1.552896e+12   28  0.016545        A01525  ...       0      0       0     1.0
80635  1.552896e+12   28  0.016545        A01525  ...       0      0       0     1.0
80636  1.552896e+12   28  0.016824 

In [ ]:
param_df

,timestamp,pid,value,device_serial,device_number,entry_time,seconds,arousal,valence,cheerful,happy,angry,nervous,sad,boredom,confusion,delight,concentration,frustration,surprise,none_1,confrustion,contempt,dejection,disgust,eureka,pride,sorrow,none_2
0,1.548121e+12,1,0.720992,A01A3A,5,1548120098000,5,0.25,0.75,1,2,1,1,1,0,0,0,1,0,0,0.0,0,0,0,0,0,0,0,1.0
1,1.548121e+12,1,0.720713,A01A3A,5,1548120098000,5,0.25,0.75,1,2,1,1,1,0,0,0,1,0,0,0.0,0,0,0,0,0,0,0,1.0
2,1.548121e+12,1,0.720341,A01A3A,5,1548120098000,5,0.25,0.75,1,2,1,1,1,0,0,0,1,0,0,0.0,0,0,0,0,0,0,0,1.0
3,1.548121e+12,1,0.719225,A01A3A,5,1548120098000,5,0.25,0.75,1,2,1,1,1,0,0,0,1,0,0,0.0,0,0,0,0,0,0,0,1.0
4,1.548121e+12,1,0.718853,A01A3A,5,1548120098000,5,0.25,0.75,1,2,1,1,1,0,0,0,1,0,0,0.0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80632,1.552896e+12,28,0.016917,A01525,1,1552894607000,640,0.50,0.75,1,1,1,1,1,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,1.0
80633,1.552896e+12,28,0.016638,A01525,1,1552894607000,640,0.50,0.75,1,1,1,1,1,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,1.0
80634,1.552896e+12,28,0.016545,A01525,1,1552894607000,640,0.50,0.75,1,1,1,1,1,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,1.0
80635,1.552896e+12,28,0.016545,A01525,1,1552894607000,640,0.50,0.75,1,1,1,1,1,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,1.0


In [ ]:
def create_info_df(df,param):
  info_df = pd.DataFrame(columns=data_cols)
  ###iterate korte hobe for starttime endtime
  i,j = 0,0
  idx = 0
  max_len = df.shape[0]
  while i < max_len and j < max_len:
    lbl_i, time_i, pid_i = df['label_negative_emotions'][i],df['timestamp'][i],df['pid'][i]
    while j < max_len and df['label_negative_emotions'][j] == lbl_i and df['pid'][j] == pid_i:
      j = j + 1
    
    new_row = {'id':'Emo_'+ str(idx), 'file_location': 'Emo_' + param + '_raw.csv', 'startTime':time_i, 'endTime':df['timestamp'][j-1], 'label': lbl_i, 'pid': 'Emo_'+ str(pid_i)}
    info_df = info_df.append(new_row, ignore_index=True)
    idx = idx + 1
    i = j
    
    
  return info_df

In [ ]:
#already preprocessed in COMBINED_RAW_DATA_FOLDER

for param in params:
  param_df = pd.read_csv(AFFECTIVE_EMO_DATA_FOLDER + param + "_raw.csv") #took EDA_raw
  windows_df = pd.read_csv(AFFECTIVE_DATA_FOLDER + param + '_Windows.csv')
  break

  #
  param_df['label_negative_emotions'] = 0
  for index, row in param_df.iterrows():
    if (row['angry'] > 1 or row['sad'] > 1 or row['nervous'] > 1):
      param_df.at[index , 'label_negative_emotions'] = 1
  info_df = create_info_df(param_df, param)    
  break    
  #info_df = create_info_df(param_df, param)
  info_df.to_csv(AFFECTIVE_EMO_DATA_FOLDER + outputfilename, index = False)
  param_df.to_csv(AFFECTIVE_DATA_FOLDER + 'Emo_' + param + "_raw.csv", index = False)

In [ ]:
#analysis
q1,q2,q3,q4 = 0,0,0,0

for index, row in eda.iterrows():
  v,a = int(eda.valence[index]),int(eda.arousal[index])
  print(v,a)
  




,A,Time,C,D,E,F,G
0,A0,B0,C0,D0,A4,C4,D4
1,A0,B0,C0,D0,A5,C5,D5
2,A2,B0,C2,D2,A4,C4,D4
3,A2,B0,C2,D2,A5,C5,D5
4,A1,B1,C1,D1,A6,C6,D6


In [ ]:
###START

In [ ]:
li_eda, li_bvp, li_temp = [],[],[]
data_directory_contents = os.listdir(AFFECTIVE_EMO_DATA_FOLDER + 'e4_data/')
for subj in data_directory_contents:
  sub = int(subj)
  label_df = pd.read_csv(AFFECTIVE_EMO_DATA_FOLDER + 'emotion_annotations/aggregated_external_annotations/' + 'P' + subj + '.external.csv')
  for param in params:
    print('WORKING-->', subj, param)
    data_df = pd.read_csv(AFFECTIVE_EMO_DATA_FOLDER + 'e4_data/' + subj + '/E4_' + param + '.csv')
    data_df['ts'] = (data_df['timestamp']/1000)
    data_df['tspan'] = (data_df['ts'] - data_df.ts[0])
    data_df['seconds'] = 0
    start, end = 0, 5 #start is the id

    for index, row in data_df.iterrows():
      t = row['tspan']
      if (t>= start and t<end):
        data_df.loc[index,'seconds'] = start
      else:
        start += 5
        end += 5  
        data_df.loc[index,'seconds'] = start

    data_df.seconds = data_df.seconds.astype(int)
    result = pd.merge(data_df, label_df, on='seconds')
    if param == 'EDA':
      li_eda.append(result)
    elif param == 'BVP':
      li_bvp.append(result)  
    else:
      li_temp.append(result)  
  #break


In [ ]:
eda = pd.concat(li_eda, axis=0, ignore_index=True) 
bvp = pd.concat(li_bvp, axis=0, ignore_index=True) 
temp = pd.concat(li_temp, axis=0, ignore_index=True) 

In [ ]:
eda.to_csv(AFFECTIVE_EMO_DATA_FOLDER + 'EDA_raw.csv', index = False)
bvp.to_csv(AFFECTIVE_EMO_DATA_FOLDER + 'BVP_raw.csv', index = False)
temp.to_csv(AFFECTIVE_EMO_DATA_FOLDER + 'TEMP_raw.csv', index = False)

In [ ]:
def get_label(v,a):
  label = 0 
  if (v>=2):
      if (a>=2):
        label = 0
      else:
        label = 1

  else:
      if (a>=2):
        label = 2
      else:      
        label = 3
  return label

In [ ]:
#another approach 

In [ ]:
data_df = data_df.drop(columns=['device_serial', 'device_number'])
data_df['ts'] = (data_df['timestamp']/1000)
data_df['tspan'] = (data_df['ts'] - data_df.ts[0])
data_df['seconds'] = 0

In [ ]:
start, end = 0, 5 #start is the id
for index, row in data_df.iterrows():
  t = row['tspan']
  if (t>= start and t<end):
    data_df.loc[index,'seconds'] = start
  else:
    start += 5
    end += 5  
    data_df.loc[index,'seconds'] = start

data_df.seconds = data_df.seconds.astype(int)

In [ ]:
result = pd.merge(data_df, label_df, on='seconds')

In [ ]:
result

,timestamp,pid,value,entry_time,ts,tspan,seconds,arousal,valence,cheerful,happy,angry,nervous,sad,boredom,confusion,delight,concentration,frustration,surprise,none_1,confrustion,contempt,dejection,disgust,eureka,pride,sorrow,none_2
0,1.548725e+12,9,0.131985,1548724984000,1.548725e+09,5.000000,5,4,4,2,2,1,1,1,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
1,1.548725e+12,9,0.129422,1548724984000,1.548725e+09,5.250000,5,4,4,2,2,1,1,1,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
2,1.548725e+12,9,0.131985,1548724984000,1.548725e+09,5.500001,5,4,4,2,2,1,1,1,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
3,1.548725e+12,9,0.133266,1548724984000,1.548725e+09,5.750000,5,4,4,2,2,1,1,1,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
4,1.548725e+12,9,0.133266,1548724984000,1.548725e+09,6.000000,5,4,4,2,2,1,1,1,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,1.548726e+12,9,0.212714,1548724984000,1.548726e+09,623.750000,620,2,4,1,1,1,1,1,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
2476,1.548726e+12,9,0.216558,1548724984000,1.548726e+09,624.000000,620,2,4,1,1,1,1,1,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
2477,1.548726e+12,9,0.213995,1548724984000,1.548726e+09,624.250000,620,2,4,1,1,1,1,1,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
2478,1.548726e+12,9,0.213995,1548724984000,1.548726e+09,624.500000,620,2,4,1,1,1,1,1,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x


In [ ]:
data_df

,timestamp,pid,value,entry_time,ts,tspan,seconds
0,1.548725e+12,9,0.000000,1548724984000,1.548725e+09,0.00,0
1,1.548725e+12,9,0.082010,1548724984000,1.548725e+09,0.25,0
2,1.548725e+12,9,0.130704,1548724984000,1.548725e+09,0.50,0
3,1.548725e+12,9,0.129422,1548724984000,1.548725e+09,0.75,0
4,1.548725e+12,9,0.129422,1548724984000,1.548725e+09,1.00,0
...,...,...,...,...,...,...,...
5948,1.548726e+12,9,0.216558,1548724984000,1.548726e+09,1487.00,1485
5949,1.548726e+12,9,0.221684,1548724984000,1.548726e+09,1487.25,1485
5950,1.548726e+12,9,0.217839,1548724984000,1.548726e+09,1487.50,1485
5951,1.548726e+12,9,0.221684,1548724984000,1.548726e+09,1487.75,1485


In [ ]:
label_df

,seconds,arousal,valence,cheerful,happy,angry,nervous,sad,boredom,confusion,delight,concentration,frustration,surprise,none_1,confrustion,contempt,dejection,disgust,eureka,pride,sorrow,none_2
0,5,4,4,2,2,1,1,1,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
1,10,4,4,2,2,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
2,15,4,4,2,2,1,1,1,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
3,20,4,4,2,2,1,1,1,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
4,25,2,4,2,1,1,1,1,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,600,2,3,2,1,1,1,1,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
120,605,2,3,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
121,610,3,3,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x
122,615,2,3,1,1,1,1,1,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x


In [ ]:
li = []
#param = "TEMP" #EDA,BVP,TEMP
for param in params:
  param_synced_data_path = SAVED_SYNCED_DATA + param + "/"
  if (os.path.isdir(param_synced_data_path) == True):
      for root, dirs, files in os.walk(param_synced_data_path):
        for file in files:
          fn = os.path.join(root,file) 
          print(fn)
          fn_only = str(Path(fn).resolve().stem)  #1_EDA
          subj_id = int(fn_only.split("_")[0])
          features_df = pd.read_csv(fn)
          li.append(features_df)

  param_df = pd.concat(li, axis=0, ignore_index=True) 

#param_df['value'] = param_df['value'] / np.max(param_df['value'])
#param_df['valence'] = param_df['valence'] / np.max(param_df['valence'])
#param_df['arousal'] = param_df['arousal'] / np.max(param_df['arousal'])

In [ ]:
eda = pd.read_csv(AFFECTIVE_EMO_DATA_FOLDER + "EDA_raw.csv") 


bvp = pd.read_csv(AFFECTIVE_EMO_DATA_FOLDER + "BVP_raw.csv") 

temp = pd.read_csv(AFFECTIVE_EMO_DATA_FOLDER + "TEMP_raw.csv") 


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,20,23,24,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,17,18,20,21,22,23,24,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
eda_w = pd.read_csv(AFFECTIVE_DATA_FOLDER + 'EDA_Windows.csv')
bvp_w = pd.read_csv(AFFECTIVE_DATA_FOLDER + 'BVP_Windows.csv')
temp_w = pd.read_csv(AFFECTIVE_DATA_FOLDER + 'TEMP_Windows.csv')

In [ ]:
def get_label(v,a):
  label = 0 
  if (v>=2):
      if (a>=2):
        label = 0
      else:
        label = 1

  else:
      if (a>=2):
        label = 2
      else:      
        label = 3
  return label

def get_index(t, df):
  for index, row in df.iterrows():
    if (row['ts'] >= t):
        return index
  return df.shape[0]

In [ ]:
max_bvp = bvp.shape[0]
max_edtemp = eda.shape[0]
ibvp, iedt = 0,0
step_bvp, step_edt = 1280, 80

while ((ibvp + step_bvp  < max_bvp) and (iedt + step_edt < max_edtemp)): #pid condition?
  #
  label_window = get_label(int(bvp.valence[ibvp]),int(bvp.arousal[ibvp]))
  subject = int(bvp.pid[ibvp])
  labelchanged = False

  for j in range(ibvp,(ibvp+step_bvp)):
    label_j = get_label(int(bvp.valence[j]),int(bvp.arousal[j]))
    subject_j = int(bvp.pid[j])
    if (label_j != label_window or subject_j!=subject):
      labelchanged = True
      break
    #

  if labelchanged == False:
    #form and append window from ib to ib+step to window dfs
    new_row = {'id':bvp_w.shape[0], 'file_location': 'Emotion/BVP_raw.csv', 'modality': 'BVP', 
               'startRowIndex':ibvp, 'endRowIndex':(ibvp+step_bvp-1), 'label': label_window, 
               'pid': 'Emo_'+ str(bvp.pid[j]), 'file_key':'value', 'application': 'Emotion'}
    bvp_w = bvp_w.append(new_row, ignore_index=True)
    new_row = {'id':eda_w.shape[0], 'file_location': 'Emotion/EDA_raw.csv', 'modality': 'EDA', 
               'startRowIndex':iedt, 'endRowIndex':(iedt+step_edt-1), 'label': label_window, 
               'pid': 'Emo_'+ str(bvp.pid[j]), 'file_key':'value', 'application': 'Emotion'}
    eda_w = eda_w.append(new_row, ignore_index=True)    
    new_row = {'id':temp_w.shape[0], 'file_location': 'Emotion/TEMP_raw.csv', 'modality': 'TEMP', 
               'startRowIndex':iedt, 'endRowIndex':(iedt+step_edt-1), 'label': label_window, 
               'pid': 'Emo_'+ str(bvp.pid[j]), 'file_key':'value', 'application': 'Emotion'}
    temp_w = temp_w.append(new_row, ignore_index=True) 

    ibvp += step_bvp
    iedt += step_edt
  else:
    ibvp = j
    iedt = get_index(bvp.ts[j],eda)

  #break
  #

In [ ]:
eda_w.reset_index(drop=True, inplace=True)
bvp_w.reset_index(drop=True, inplace=True)
temp_w.reset_index(drop=True, inplace=True)
eda_w['id'] = eda_w.index
bvp_w['id'] = bvp_w.index
temp_w['id'] = temp_w.index

In [ ]:
eda_w.to_csv(AFFECTIVE_DATA_FOLDER + 'EDA_W.csv', index = False)
bvp_w.to_csv(AFFECTIVE_DATA_FOLDER + 'BVP_W.csv', index = False)
temp_w.to_csv(AFFECTIVE_DATA_FOLDER + 'TEMP_W.csv', index = False)

In [ ]:
eda_w = pd.read_csv(AFFECTIVE_DATA_FOLDER + 'EDA_W.csv')
bvp_w = pd.read_csv(AFFECTIVE_DATA_FOLDER + 'BVP_W.csv')
temp_w = pd.read_csv(AFFECTIVE_DATA_FOLDER + 'TEMP_W.csv')

In [ ]:
df_emo = eda_w[eda_w['application'] == 'Emotion']
print("<---Emotion overall data samples count-->", df_emo.shape[0])
print("Class 0 count =", (df_emo[df_emo['label'] == 0]).shape[0])
print("Class 1 count =", (df_emo[df_emo['label'] == 1]).shape[0])
print("Class 2 count =", (df_emo[df_emo['label'] == 2]).shape[0])
print("Class 3 count =", (df_emo[df_emo['label'] == 3]).shape[0])

<---Emotion overall data samples count--> 723
Class 0 count = 466
Class 1 count = 243
Class 2 count = 12
Class 3 count = 2


In [ ]:
print(temp_w.shape,bvp_w.shape,eda_w.shape)

(10166, 9) (10166, 9) (10166, 9)


In [ ]:
######END

In [ ]:
#IDX RENEW

In [ ]:
#0 = cheerful happy (PVHA)
#1 = angry nervous (NVHA)
#1 = sad (NVLA)

for index, row in param_df.iterrows():
  #process_row(row)
  print(row)
  label = get_label(int(row['valence']),int(row['arousal']))

  i,j = 0,0
  idx = windows_df.shape[0]
  max_len = param_df.shape[0]
  while i < max_len and j < max_len:
    lbl_i, pid_i = param_df['label'][i],param_df['pid'][i]
    while j < max_len and param_df['label'][j] == lbl_i and param_df['pid'][j] == pid_i:
      j = j + 1
    
    new_row = {'id': idx, 'file_location': 'Emotion/' + param + "_raw.csv", 'startRowIndex':i, 'endRowIndex':(j-1), 'label': int(lbl_i), 'pid': 'Emotion_'+ str(int(pid_i))}
    info_df = info_df.append(new_row, ignore_index=True)
    idx = idx + 1
    i = j
  #PVHA = (int(row['cheerful']) + int(row['happy'])) / 2
  #NVHA = (int(row['angry']) + int(row['nervous'])) / 2
  #NVLA = (int(row['sad']))
  #print(PVHA,NVHA,NVLA)
  #ls = [PVHA,NVHA,NVLA]
  #print(max(ls))  
  break

timestamp          1.54812e+12
pid                          1
value                 0.720992
device_serial           A01A3A
device_number                5
entry_time       1548120098000
seconds                      5
arousal                   0.25
valence                   0.75
cheerful                     1
happy                        2
angry                        1
nervous                      1
sad                          1
boredom                      0
confusion                    0
delight                      0
concentration                1
frustration                  0
surprise                     0
none_1                       0
confrustion                  0
contempt                     0
dejection                    0
disgust                      0
eureka                       0
pride                        0
sorrow                       0
none_2                       1
Name: 0, dtype: object


NameError: ignored

In [ ]:
  param_df['label_negative_emotions'] = 0
  count_p = 0
  count_n = 0
  for index, row in param_df.iterrows():
    if (row['angry'] > 1 or row['sad'] > 1 or row['nervous'] > 1):
      count_p = count_p + 1
      param_df.at[index , 'label_negative_emotions'] = 1
    else:
      count_n = count_n + 1

  print(count_p,count_n)    

8915 71722


In [ ]:
param_df.label_negative_emotions.unique()

array([0])

In [ ]:
param_df.nervous.unique()

NameError: ignored

In [ ]:
param_df['lbl'] = np.where((param_df['angry'] >= 2 or param_df['sad'] >= 2 or param_df['nervous'] >= 2), True, False)

ValueError: ignored

In [ ]:
info_df = create_info_df(param_df, param) 

In [ ]:
info_df

,id,file_location,startTime,endTime,label,pid
0,Emo_0,Emo_EDA_raw.csv,1.548121e+12,1.548121e+12,0,Emo_1
1,Emo_1,Emo_EDA_raw.csv,1.548121e+12,1.548121e+12,1,Emo_1
2,Emo_2,Emo_EDA_raw.csv,1.548121e+12,1.548121e+12,0,Emo_1
3,Emo_3,Emo_EDA_raw.csv,1.548121e+12,1.548121e+12,1,Emo_1
4,Emo_4,Emo_EDA_raw.csv,1.548121e+12,1.548122e+12,0,Emo_1
...,...,...,...,...,...,...
183,Emo_183,Emo_EDA_raw.csv,1.552896e+12,1.552896e+12,0,Emo_28
184,Emo_184,Emo_EDA_raw.csv,1.552896e+12,1.552896e+12,1,Emo_28
185,Emo_185,Emo_EDA_raw.csv,1.552896e+12,1.552896e+12,0,Emo_28
186,Emo_186,Emo_EDA_raw.csv,1.552896e+12,1.552896e+12,1,Emo_28


In [ ]:
info_df.shape

(28, 9)

In [ ]:
param_df.keys

<bound method NDFrame.keys of           timestamp  pid     value  ... sorrow  none_2  label_negative_emotions
0      1.548121e+12    1  0.720992  ...      0     1.0                        0
1      1.548121e+12    1  0.720713  ...      0     1.0                        0
2      1.548121e+12    1  0.720341  ...      0     1.0                        0
3      1.548121e+12    1  0.719225  ...      0     1.0                        0
4      1.548121e+12    1  0.718853  ...      0     1.0                        0
...             ...  ...       ...  ...    ...     ...                      ...
80632  1.552896e+12   28  0.016917  ...      0     1.0                        0
80633  1.552896e+12   28  0.016638  ...      0     1.0                        0
80634  1.552896e+12   28  0.016545  ...      0     1.0                        0
80635  1.552896e+12   28  0.016545  ...      0     1.0                        0
80636  1.552896e+12   28  0.016824  ...      0     1.0                        0

[80637 ro

In [ ]:
  for index, row in param_df.iterrows():
    if (row['angry'] >= 2 or row['sad'] >= 2 or row['nervous'] >= 2):
      print(row.label_negative_emotions)

In [ ]:
info_df = pd.DataFrame(columns=data_cols)
i = 0
idx = 0
max_len = param_df.shape[0]
max_len

80637

In [ ]:
param_df.label_negative_emotions.unique()

array([0])

In [ ]:
param_df['value'][0]

0.7209919317397288

In [ ]:
param_df

,timestamp,pid,value,device_serial,device_number,entry_time,seconds,arousal,valence,cheerful,happy,angry,nervous,sad,boredom,confusion,delight,concentration,frustration,surprise,none_1,confrustion,contempt,dejection,disgust,eureka,pride,sorrow,none_2,label_negative_emotions
0,1.548121e+12,1,0.720992,A01A3A,5,1548120098000,5,0.25,0.75,1,2,1,1,1,0,0,0,1,0,0,0.0,0,0,0,0,0,0,0,1.0,0
1,1.548121e+12,1,0.720713,A01A3A,5,1548120098000,5,0.25,0.75,1,2,1,1,1,0,0,0,1,0,0,0.0,0,0,0,0,0,0,0,1.0,0
2,1.548121e+12,1,0.720341,A01A3A,5,1548120098000,5,0.25,0.75,1,2,1,1,1,0,0,0,1,0,0,0.0,0,0,0,0,0,0,0,1.0,0
3,1.548121e+12,1,0.719225,A01A3A,5,1548120098000,5,0.25,0.75,1,2,1,1,1,0,0,0,1,0,0,0.0,0,0,0,0,0,0,0,1.0,0
4,1.548121e+12,1,0.718853,A01A3A,5,1548120098000,5,0.25,0.75,1,2,1,1,1,0,0,0,1,0,0,0.0,0,0,0,0,0,0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80632,1.552896e+12,28,0.016917,A01525,1,1552894607000,640,0.50,0.75,1,1,1,1,1,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,1.0,0
80633,1.552896e+12,28,0.016638,A01525,1,1552894607000,640,0.50,0.75,1,1,1,1,1,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,1.0,0
80634,1.552896e+12,28,0.016545,A01525,1,1552894607000,640,0.50,0.75,1,1,1,1,1,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,1.0,0
80635,1.552896e+12,28,0.016545,A01525,1,1552894607000,640,0.50,0.75,1,1,1,1,1,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,1.0,0


In [ ]:
eda_windows_df

In [ ]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "Time": ["B0", "B1", "B0", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

df2 = pd.DataFrame(
    {
        "E": ["A4", "A5", "A6", "A7"],
        "Time": ["B0", "B0", "B1", "B7"],
        "F": ["C4", "C5", "C6", "C7"],
        "G": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)

result = pd.merge(df1, df2, on='Time') #pd.concat([df1, df2], axis=0)
result

,A,Time,C,D,E,F,G
0,A0,B0,C0,D0,A4,C4,D4
1,A0,B0,C0,D0,A5,C5,D5
2,A2,B0,C2,D2,A4,C4,D4
3,A2,B0,C2,D2,A5,C5,D5
4,A1,B1,C1,D1,A6,C6,D6


In [ ]:
#EMO CONFIDENCE

eda_stress = pd.read_csv(AFFECTIVE_DATA_FOLDER + 'EDA_raw.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,20,23,24,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
  param_synced_data_path = AFFECTIVE_DATA_FOLDER + 'Stress1'
  if (os.path.isdir(param_synced_data_path) == True):
      for root, dirs, files in os.walk(param_synced_data_path):
        for file in files:
          fn = os.path.join(root,file) 
          print(fn)
          fn_only = str(Path(fn).resolve().stem)  #1_EDA
          subj_id = int(fn_only.split("_")[0])
          features_df = pd.read_csv(fn)
          li.append(features_df)

  param_df = pd.concat(li, axis=0, ignore_index=True) 

In [ ]:
import os, sys, glob
li = []
os.chdir(AFFECTIVE_DATA_FOLDER + 'Stress1/')
for file in glob.glob("*.csv"):
  if file.startswith('EDA'):
    print(file)
    df = pd.read_csv(file)
    li.append(df)

eda_stress = pd.concat(li, axis=0, ignore_index=True) 

EDA_2.csv
EDA_3.csv
EDA_4.csv
EDA_5.csv
EDA_6.csv
EDA_7.csv
EDA_8.csv
EDA_9.csv
EDA_11.csv
EDA_13.csv
EDA_14.csv
EDA_15.csv
EDA_16.csv
EDA_17.csv
EDA_10.csv


In [ ]:
eda_stress

,pid,value,label
0,2.0,0.010851,1.0
1,2.0,0.010851,1.0
2,2.0,0.010851,1.0
3,2.0,0.010661,1.0
4,2.0,0.010851,1.0
...,...,...,...
86935,10.0,0.085020,3.0
86936,10.0,0.085020,3.0
86937,10.0,0.085058,3.0
86938,10.0,0.085172,3.0


In [ ]:
model = Sequential()
model.add(Dense(25, input_dim=1, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 25)                50        
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 26        
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#this right
model = Sequential()
model.add(Dense(10, input_dim=1, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 10)                20        
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 11        
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_df, test_df = train_test_split(eda_stress,shuffle=True,test_size=0.2)
trainX, trainY = train_df['value'],train_df['label']
testX, testY = train_df['value'],train_df['label']
model.fit(trainX, trainY, batch_size=32, epochs=10)

Epoch 1/10
2174/2174 [==============================] - 4s 2ms/step - loss: -4.3199 - accuracy: 0.8088
Epoch 2/10
2174/2174 [==============================] - 3s 1ms/step - loss: -27.5419 - accuracy: 0.8094
Epoch 3/10
2174/2174 [==============================] - 3s 2ms/step - loss: -67.0484 - accuracy: 0.8094
Epoch 4/10
2174/2174 [==============================] - 3s 1ms/step - loss: -120.2190 - accuracy: 0.8094
Epoch 5/10
2174/2174 [==============================] - 3s 1ms/step - loss: -186.4475 - accuracy: 0.8094
Epoch 6/10
2174/2174 [==============================] - 3s 1ms/step - loss: -265.0651 - accuracy: 0.8094
Epoch 7/10
2174/2174 [==============================] - 3s 2ms/step - loss: -356.5673 - accuracy: 0.8094
Epoch 8/10
2174/2174 [==============================] - 3s 1ms/step - loss: -460.5486 - accuracy: 0.8094
Epoch 9/10
2174/2174 [==============================] - 3s 1ms/step - loss: -577.3118 - accuracy: 0.8094
Epoch 10/10
2174/2174 [==============================] - 3s

In [ ]:
model_output = model.get_layer("dense_27").output
m = Model(inputs=model.input, outputs=model_output)
outp = m.predict(testX)
print(testX.shape, outp.shape, outp)

(69552,) (69552, 10) [[22.810986  0.       23.188616 ...  0.        0.        0.      ]
 [25.267027  0.       25.722555 ...  0.        0.        0.      ]
 [27.926624  0.       28.466507 ...  0.        0.        0.      ]
 ...
 [22.835794  0.       23.21421  ...  0.        0.        0.      ]
 [22.739252  0.       23.114607 ...  0.        0.        0.      ]
 [22.818003  0.       23.195856 ...  0.        0.        0.      ]]


In [ ]:
outp[1]

array([25.267027,  0.      , 25.722555, 25.190971,  0.      , 25.49633 ,
        0.      ,  0.      ,  0.      ,  0.      ], dtype=float32)

In [ ]:
#l = list(testX)
#print(l)
arr = np.array([l[0]])
print(arr)
arr[0].shape

[0.00723737]


()

In [ ]:
outp = m.predict(arr)
outp

array([[22.810986,  0.      , 23.188616, 22.746313,  0.      , 22.983826,
         0.      ,  0.      ,  0.      ,  0.      ]], dtype=float32)

In [ ]:
def mahalanobis_distance_from_class(test_sample_vec, class_vec_array): 
  mean = np.mean(class_vec_array, axis=0) 
  cov = np.cov(class_vec_array.T)
  cov_inv = np.linalg.inv(cov)
  diff = test_sample_vec - mean

  md = np.dot(diff.T, cov_inv)
  md = np.dot(md, diff) #md = np.dot(md, diff.T)
  md = np.sqrt(md) #np.sqrt(md.diagonal())

  return md

def MahalanobisDist(inv_cov_matrix, mean_distr, data, verbose=False):
    inv_covariance_matrix = inv_cov_matrix
    vars_mean = mean_distr
    diff = data - vars_mean
    md = []
    for i in range(len(diff)):
        md.append(np.sqrt(diff[i].dot(inv_covariance_matrix).dot(diff[i])))
    return md

In [ ]:
class_vec_array = np.array([
[64.0, 580.0, 29.0],    
[66.0, 570.0, 33.0], 
[68.0, 590.0, 37.0], 
[69.0, 660.0, 46.0],
[73.0, 600.0, 55.0],
])

class_vec_array

array([[ 64., 580.,  29.],
       [ 66., 570.,  33.],
       [ 68., 590.,  37.],
       [ 69., 660.,  46.],
       [ 73., 600.,  55.]])

In [ ]:
test_sample_vec = np.array([50,600,30])
test_sample_vec

array([ 50, 600,  30])

In [ ]:
mahalanobis_distance_from_class(test_sample_vec, class_vec_array)

27.917472072005943